In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import numpy as np
import pandas as pd
from scipy import stats
from matplotlib import pyplot as plt
import seaborn as sns
import statsmodels.formula.api as smf
import statsmodels.api as sm
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori, association_rules
import warnings
warnings.filterwarnings('ignore')
import missingno
from sklearn.preprocessing import LabelEncoder
import re
from sklearn.metrics import accuracy_score

from matplotlib import font_manager, rc
font_path = "/content/drive/MyDrive/L1/Notebooks/dataset/malgun.ttf"
font_name = font_manager.FontProperties(fname=font_path).get_name()
rc('font', family=font_name)

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


- cloth, hobby share 증감 독립변수

In [ ]:
# cloth, hobby 세부 카테고리 증감률 계산을 포함한 df read
df = pd.read_pickle('/content/drive/MyDrive/L1/Notebooks/dataset/X_cloth_hobby_var.pkl')
df.head()

,CUSTNO,spent_less,cloth_men_purch_diff,cloth_kids_purch_diff,cloth_casual_purch_diff,cloth_lingerie_purch_diff,cloth_sports_purch_diff,cloth_etc_purch_diff,cloth_business_purch_diff,hobby_media_purch_diff,hobby_child_purch_diff,hobby_pet_purch_diff,hobby_outdoor_purch_diff,hobby_sports_purch_diff,hobby_health_purch_diff,hobby_instex_purch_diff,cloth_women_purch_diff,hobby_golf_purch_diff
0,00001,0.0,-0.000900,0.0,0.000137,0.000000,0.0,-0.000052,-0.000102,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.000000
1,00002,1.0,0.000000,0.0,0.000090,0.001966,0.0,-0.000142,0.000000,0.0,0.0,-0.00036,0.0,0.0,0.0,0.0,0.0,-0.000468
2,00004,0.0,-0.000028,0.0,-0.000032,0.000000,0.0,0.000000,0.000253,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,-0.000075
3,00006,1.0,0.000071,0.0,0.000038,0.000348,0.0,0.000037,-0.000011,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.000012
4,00007,1.0,-0.000100,0.0,0.000680,0.000114,0.0,-0.000813,-0.000187,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,-0.000003


In [ ]:
df.cloth_men_purch_diff.value_counts()

 0.000000    12792
-0.000100      410
-0.000200      237
-0.000300      161
-0.000400      119
             ...  
 0.001695        1
 0.002325        1
-0.000407        1
-0.000800        1
 0.000259        1
Name: cloth_men_purch_diff, Length: 1937, dtype: int64

In [ ]:
ddf = df.copy()

In [ ]:
afill_diff = pd.read_pickle('/content/drive/MyDrive/L1/Notebooks/dataset/X_Afill_var.pkl')
ddf = pd.merge(ddf, afill_diff, on='CUSTNO', how='right').fillna(0)

- Recency 독립변수

In [ ]:
Recency = pd.read_pickle('/content/drive/MyDrive/L1/Notebooks/dataset/X_recency.pkl')
ddf = pd.merge(ddf, Recency, on='CUSTNO', how='right').fillna(0)

- membership 독립변수

In [ ]:
membership = pd.read_pickle('/content/drive/MyDrive/L1/Notebooks/dataset/membership.pkl')
ddf = pd.merge(ddf, 
                 membership[['CUSTNO','MEMBERSHIP_CAT']], 
                    on='CUSTNO', how='left').fillna(0)

- Value_categorize 독립변수
    - 고,중,저가 제품 구매빈도 변동

In [ ]:
value_cat2 = pd.read_pickle('/content/drive/MyDrive/L1/Notebooks/dataset/categorize_value_abs.pkl')
ddf = pd.merge(ddf, 
                 value_cat2, on='CUSTNO', how='left').fillna(0)

- visit_per_day
    - 일 방문횟수 변동

In [ ]:
visit_per_day = pd.read_pickle('/content/drive/MyDrive/L1/Notebooks/dataset/PERDAY_VISITING.pkl')
visit_per_day
ddf = pd.merge(ddf, visit_per_day, on='CUSTNO', how='left').fillna(0)

- 고객속성 군집분석 독립변수
    - 성별, 연령대, 지역, 구매금액으로 나눈 고객그룹

In [ ]:
cust_class = pd.read_pickle('dataset/cluster_label.pkl')
ddf.CUSTNO = ddf.CUSTNO.astype(int)
ddf = pd.merge(ddf, 
                 cust_class, on='CUSTNO', how='left').fillna(0)

FileNotFoundError: ignored

In [ ]:
ddf.head()

,CUSTNO,spent_less,cloth_men_purch_diff,cloth_kids_purch_diff,cloth_casual_purch_diff,cloth_lingerie_purch_diff,cloth_sports_purch_diff,cloth_etc_purch_diff,cloth_business_purch_diff,hobby_media_purch_diff,hobby_child_purch_diff,hobby_pet_purch_diff,hobby_outdoor_purch_diff,hobby_sports_purch_diff,hobby_health_purch_diff,hobby_instex_purch_diff,cloth_women_purch_diff,hobby_golf_purch_diff,A_purch_diff,B_purch_diff,Recency,MEMBERSHIP_CAT,cloth_highvalue,cloth_lowvalue,cloth_mediumvalue,hobby_highvalue,hobby_lowvalue,hobby_mediumvalue,CLOTH_MEN_PERDAY_VISITING,CLOTH_CASUAL_PERDAY_VISITING,CLOTH_ETC_PERDAY_VISITING,CLOTH_BUSINESS_PERDAY_VISITING,CLOTH_LINGERIE_PERDAY_VISITING,CLOTH_SPORTS_PERDAY_VISITING,CLOTH_KIDS_PERDAY_VISITING,CLOTH_WOMEN_PERDAY_VISITING,HOBBY_PET_PERDAY_VISITING,HOBBY_GOLF_PERDAY_VISITING,HOBBY_MEDIA_PERDAY_VISITING,HOBBY_CHILD_PERDAY_VISITING,HOBBY_HEALTH_PERDAY_VISITING,HOBBY_OUTDOOR_PERDAY_VISITING,HOBBY_SPORTS_PERDAY_VISITING,HOBBY_INSTEX_PERDAY_VISITING
0,00001,0.0,-0.000900,0.0,0.000137,0.000000,0.0,-0.000052,-0.000102,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.000000,-0.000050,0.000000,29,0,3.0,0.0,0.0,0.0,0.0,0.0,-0.0659,0.1099,-0.0110,0.000,0.0000,0.0,0.0,0.0,0.000,0.0000,0.0,0.0,0.0,0.0,0.0,0.0
1,00002,1.0,0.000000,0.0,0.000090,0.001966,0.0,-0.000142,0.000000,0.0,0.0,-0.00036,0.0,0.0,0.0,0.0,0.0,-0.000468,0.000032,0.000000,2,0,2.0,0.0,0.0,5.0,2.0,0.0,0.0000,0.0330,-0.0330,0.000,0.0219,0.0,0.0,0.0,-0.011,-0.0549,0.0,0.0,0.0,0.0,0.0,0.0
2,00004,0.0,-0.000028,0.0,-0.000032,0.000000,0.0,0.000000,0.000253,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,-0.000075,0.000025,0.000046,9,0,1.0,0.0,0.0,1.0,0.0,0.0,0.0000,-0.0439,0.0000,0.033,0.0000,0.0,0.0,0.0,0.000,-0.0110,0.0,0.0,0.0,0.0,0.0,0.0
3,00006,1.0,0.000071,0.0,0.000038,0.000348,0.0,0.000037,-0.000011,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.000012,0.000058,0.000050,25,0,12.0,0.0,4.0,1.0,0.0,0.0,0.0110,0.0879,0.0219,-0.011,0.0659,0.0,0.0,0.0,0.000,0.0110,0.0,0.0,0.0,0.0,0.0,0.0
4,00007,1.0,-0.000100,0.0,0.000680,0.000114,0.0,-0.000813,-0.000187,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,-0.000003,0.000233,0.000000,2,0,5.0,0.0,0.0,1.0,0.0,0.0,-0.0110,0.0440,-0.0769,-0.022,0.0110,0.0,0.0,0.0,0.000,-0.0110,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
ddf.columns

Index(['CUSTNO', 'spent_less', 'cloth_men_purch_diff', 'cloth_kids_purch_diff',
       'cloth_casual_purch_diff', 'cloth_lingerie_purch_diff',
       'cloth_sports_purch_diff', 'cloth_etc_purch_diff',
       'cloth_business_purch_diff', 'hobby_media_purch_diff',
       'hobby_child_purch_diff', 'hobby_pet_purch_diff',
       'hobby_outdoor_purch_diff', 'hobby_sports_purch_diff',
       'hobby_health_purch_diff', 'hobby_instex_purch_diff',
       'cloth_women_purch_diff', 'hobby_golf_purch_diff', 'A_purch_diff',
       'B_purch_diff', 'Recency', 'MEMBERSHIP_CAT', 'cloth_highvalue',
       'cloth_lowvalue', 'cloth_mediumvalue', 'hobby_highvalue',
       'hobby_lowvalue', 'hobby_mediumvalue', 'CLOTH_MEN_PERDAY_VISITING',
       'CLOTH_CASUAL_PERDAY_VISITING', 'CLOTH_ETC_PERDAY_VISITING',
       'CLOTH_BUSINESS_PERDAY_VISITING', 'CLOTH_LINGERIE_PERDAY_VISITING',
       'CLOTH_SPORTS_PERDAY_VISITING', 'CLOTH_KIDS_PERDAY_VISITING',
       'CLOTH_WOMEN_PERDAY_VISITING', 'HOBBY_PET_PERDAY_VISI

In [ ]:
ddf.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 16295 entries, 0 to 16294
Data columns (total 44 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   CUSTNO                          16295 non-null  object 
 1   spent_less                      16295 non-null  float64
 2   cloth_men_purch_diff            16295 non-null  float64
 3   cloth_kids_purch_diff           16295 non-null  float64
 4   cloth_casual_purch_diff         16295 non-null  float64
 5   cloth_lingerie_purch_diff       16295 non-null  float64
 6   cloth_sports_purch_diff         16295 non-null  float64
 7   cloth_etc_purch_diff            16295 non-null  float64
 8   cloth_business_purch_diff       16295 non-null  float64
 9   hobby_media_purch_diff          16295 non-null  float64
 10  hobby_child_purch_diff          16295 non-null  float64
 11  hobby_pet_purch_diff            16295 non-null  float64
 12  hobby_outdoor_purch_diff        

In [ ]:
print('get_dummies() 수행 전 데이터 Shape:', ddf.shape)
ddf = pd.get_dummies(ddf)
print('get_dummies() 수행 후 데이터 Shape:', ddf.shape)

get_dummies() 수행 전 데이터 Shape: (16295, 44)
get_dummies() 수행 후 데이터 Shape: (16295, 16341)


In [ ]:
corr = ddf.corr()
mask = np.zeros_like(corr, dtype=np.bool)
mask[np.triu_indices_from(mask)]= True
plt.figure(figsize=(30,20))
sns.heatmap(corr, mask=mask, annot=False)

In [ ]:
ad = ddf.corr()
ad.style.background_gradient(cmap = 'coolwarm')

- 독립변수끼리 상관관계가 높은 columns 제거 (상관관계 0.7이상 1미만)

In [ ]:
ddf.drop(['cloth_casual_purch_diff',
          'hobby_child_purch_diff',
          'MEMBERSHIP_CAT_0',
          'MEMBERSHIP_CAT_1',
          'HOBBY_SPORTS_PERDAY_VISITING',
          'CLOTH_KIDS_PERDAY_VISITING',
          'cloth_men_purch_diff',
          'cloth_kids_purch_diff',
          'HOBBY_SPORTS_PERDAY_VISITING'],axis=1,inplace=True)

In [ ]:
ddf.to_pickle('/content/drive/MyDrive/L1/Notebooks/dataset/ddf_with_CUSTNO.pkl')

In [ ]:
ddf.drop(['CUSTNO'], axis=1, inplace=True)

In [ ]:
ddf.to_pickle('/content/drive/MyDrive/L1/Notebooks/dataset/dataset_for_analysis.pkl')